# Initialization and loading basic functions  

In [34]:
import os
import numpy as np
import codecs
import json
import string
import random
import itertools
from datetime import datetime, date, timedelta
import mysql.connector
from mysql.connector import errorcode

DB_NAME = "TaxiTest"
config = {
        'user': 'root',
        'password': '9090',
        'host': '127.0.0.1',
        'database': 'taxitest',
        'raise_on_warnings': True,
        'use_pure': False
}

def get_table_review(table, only_columns_names = False):
    # print(table)
    # print("Field, Type, Null, Key, Default, Extra")
    columns_names = []

    q = "DESCRIBE " + table
    cursor.execute(q)
    columns_attributes = cursor.fetchall()
    for column_attributes_tuple in columns_attributes:
        column_attributes = list(column_attributes_tuple)
        column_attributes[1] = str(column_attributes[1])[2:-1]
        columns_names.append(column_attributes[0])
        if not only_columns_names:
            print(', '.join([str(a) for a in column_attributes]))
    
    joined = ', '.join(columns_names)
    joined_values = ', '.join('%s' for c in columns_names)
    print("Columns: (", joined, ")")
    if not only_columns_names:
        insert_string = "query = \"INSERT INTO %s (%s) VALUES (%s)\"" % (table, joined, joined_values)
        print(insert_string)
    
def count_rows(table):
    q = "SELECT COUNT(*) FROM " + table
    cursor.execute(q)
    log = cursor.fetchall()
    print("Total number of rows %s: %s." % (table, log))
    return int(log[0][0])

def delete_all_rows(table):
    q = "DELETE FROM " + table
    cursor.execute(q) 
    
def select_rows(q, table=None):
    cursor.execute(q)
    return cursor.fetchall()

def insert_many_rows(q, data, table=None):
    try:
        cursor.executemany(q, data)
        log = cursor.fetchall()
        print("Result: " + log)
    except mysql.connector.Error as err:
        print(err.msg)

cnx = mysql.connector.connect(**config)
cnx.autocommit = True
cursor = cnx.cursor()

query = "SHOW DATABASES"
cursor.execute(query)
rows = cursor.fetchall()
print("Total number of databases:", cursor.rowcount)
for row in rows:
    print(row)
    
cursor.execute("USE TaxiTest")

Total number of databases: 7
('information_schema',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('taxitest',)
('world',)


In [2]:
# DESCRIBE taxitest.order;

cursor.execute("USE " + DB_NAME)
cursor.execute("SHOW Tables")
tables = cursor.fetchall()


for table in tables:
    print("Table:", str(table[0]))
    get_table_review(table[0], only_columns_names=True)


Table: address
Columns: ( idAddress, City, Street, HouseNumber )
Table: car
Columns: ( idCar, Lisense, Mark, Model, Insurance_idInsurance, ChildSeat )
Table: client
Columns: ( idClient, Name, Phone, Password )
Table: driver
Columns: ( idDriver, Name, DriverLisence, Phone, isWorking )
Table: insurance
Columns: ( idInsurance, InsCompany_idInsCompany, InsuranceStartDate, InsuranceEndDate )
Table: insurancecompany
Columns: ( idInsuranceCompany, Address, Phone, Mail, Name )
Table: operator
Columns: ( idOperator, Name, Phone, Mail )
Table: orders
Columns: ( idOrder, Call_idCall, Departure_idAddress, Destination_idAddress, Taxi_idTaxi, Payment_idPayment, StartDatetime, FinishDatetime, Cost, Rate )
Table: payment
Columns: ( idPayment, PayTime, PaymentType, PaymentService_idPaymentService, Paycheck )
Table: paymentservice
Columns: ( idPaymentService, Name, Address, Site )
Table: phonecall
Columns: ( idCall, Operator_idOperator, Client_idClient, datetimeValue, Duration )
Table: taxi
Columns: ( i

# Start

## Table `insurancecompany`

In [3]:
table = DB_NAME + "." + "insurancecompany"
get_table_review(table)

idInsuranceCompany, int(11), NO, PRI, None, 
Address, varchar(45), NO, , None, 
Phone, varchar(45), NO, , None, 
Mail, varchar(45), NO, , None, 
Name, varchar(45), NO, , None, 
Columns: ( idInsuranceCompany, Address, Phone, Mail, Name )
query = "INSERT INTO TaxiTest.insurancecompany (idInsuranceCompany, Address, Phone, Mail, Name) VALUES (%s, %s, %s, %s, %s)"


In [5]:
count_rows(table)

Total number of rows TaxiTest.insurancecompany: [(30,)].


30

In [12]:
# Data generation

with open('InsuranceCompany/Company.json') as f:
    names = json.load(f)

total_number = len(names)
print("len(names): ", len(names))
idInsuranceCompany = np.arange(total_number)

with open('Street\Street.json') as f:
    streets = json.load(f)
    streets = np.random.choice(streets, total_number, replace=False)

query = "INSERT INTO insurancecompany (idInsuranceCompany, Address, Phone, Mail, Name) VALUES (%s, %s, %s, %s, %s)"
data = []

for i in range(total_number):
    idInsuranceCompany = i + 1
    address = "Kazan, " + streets[i] + ", " + str(np.random.randint(1, 200))
    phone = '+7(843)' + '{:07}'.format(np.random.randint(0, 9999999))
    name = names[i]
    mail = name.lower().replace(" ", "") + '@gmail.com'
    data.append((idInsuranceCompany, address, phone, mail, name))

print("Q: ", query)
print("D: ", data[0])

# insert_many_rows(query, data)

len(names):  49
Q:  INSERT INTO insurancecompany (idInsuranceCompany, Address, Phone, Mail, Name) VALUES (%s, %s, %s, %s, %s)
D:  (1, 'Kazan, Batyrlyk, 98', '+7(843)6297134', 'asko@gmail.com', 'ASKO')


## Table `insurance`

*** Changes ***
~~~~python
# Modify type
cursor.execute("ALTER TABLE " + table + " MODIFY InsuranceStartDate DATE")
cursor.execute("ALTER TABLE " + table + " MODIFY InsuranceEndDate DATE")

# Change order
cursor.execute("ALTER TABLE " + table + " CHANGE COLUMN InsuranceEndDate InsuranceEndDate Date AFTER InsuranceStartDate")
~~~~

In [16]:
table = DB_NAME + "." + "insurance"
get_table_review(table)

idInsurance, int(11), NO, PRI, None, 
InsCompany_idInsCompany, int(11), NO, MUL, None, 
InsuranceStartDate, date, YES, , None, 
InsuranceEndDate, date, YES, , None, 
Columns: ( idInsurance, InsCompany_idInsCompany, InsuranceStartDate, InsuranceEndDate )
query = "INSERT INTO TaxiTest.insurance (idInsurance, InsCompany_idInsCompany, InsuranceStartDate, InsuranceEndDate) VALUES (%s, %s, %s, %s)"


In [17]:
count_rows(table)

Total number of rows TaxiTest.insurance: [(100,)].


100

In [18]:
# Data generation
with open('InsuranceCompany\Company.json', 'w') as outfile:
    json.dump(names, outfile)

query = "INSERT INTO TaxiTest.insurance (idInsurance, InsCompany_idInsCompany, InsuranceEndDate, InsuranceStartDate) VALUES (%s , %s , %s , %s )"
data = []

total_number = 100

for i in range(total_number):
    idInsurance = i + 1 
    InsCompany_idInsCompany = np.random.randint(1, 30)
    InsuranceStartDate = date(2014 + np.random.randint(-3, 3), np.random.randint(1, 10), np.random.randint(1, 25))
    InsuranceEndDate = InsuranceStartDate + timedelta(days=25*int(np.random.choice(np.array([6, 8, 12]))))
    data.append((idInsurance, InsCompany_idInsCompany, InsuranceStartDate, InsuranceEndDate))
    #print(data[i])
    
print("Q: ", query)
print("D: ", data[0])

# insert_many_rows(query, data)

Q:  INSERT INTO TaxiTest.insurance (idInsurance, InsCompany_idInsCompany, InsuranceEndDate, InsuranceStartDate) VALUES (%s , %s , %s , %s )
D:  (1, 29, datetime.date(2016, 6, 19), datetime.date(2017, 1, 5))


## Table `car`

*** Changes ***
~~~~python
# Modify type
cursor.execute("ALTER TABLE " + table + " MODIFY ChildSeat INT")
~~~~


In [19]:
table = DB_NAME + "." + "car"
get_table_review(table)
count_rows(table)

idCar, int(11), NO, PRI, None, 
Lisense, varchar(45), NO, , None, 
Mark, varchar(45), NO, , None, 
Model, varchar(45), NO, , None, 
Insurance_idInsurance, int(11), NO, MUL, None, 
ChildSeat, int(11), YES, , None, 
Columns: ( idCar, Lisense, Mark, Model, Insurance_idInsurance, ChildSeat )
query = "INSERT INTO TaxiTest.car (idCar, Lisense, Mark, Model, Insurance_idInsurance, ChildSeat) VALUES (%s, %s, %s, %s, %s, %s)"
Total number of rows TaxiTest.car: [(100,)].


100

In [20]:
# Data generation
query = "INSERT INTO TaxiTest.car (idCar, Lisense, Mark, Model, Insurance_idInsurance, ChildSeat) VALUES (%s, %s, %s, %s, %s, %s)"
data = []

total_number = 100
# Get insurances (better in random order), we have 100 insurances -> total_number = 100 of cars available
# Warning: np.arange(a) return from 0 to a - 1
# Writing Insurance_idInsurance which doesn't exist in insurance is restricted - QUERY will not be executed
insurances = np.arange(1, total_number + 1)
np.random.shuffle(insurances)

# Load cars
with open("Car/Car.json") as f:
    cars = json.load(f)

# Generate UNIQUE car licenses
licenses = set()
alphanums = string.ascii_uppercase + string.digits
while len(licenses) != 100:
    licenses.add(''.join(random.choice(alphanums) for _ in range(7)))
licenses = list(licenses)

for i in range(total_number):
    idCar = i + 1 
    Lisense = licenses[i]
    (Mark, Model) = np.random.choice(cars).split(' ')
    # Failed processing format-parameters; Python 'int32' cannot be converted to a MySQL type
    Insurance_idInsurance = int(insurances[i])
    ChildSeat = int(np.random.choice([0, 1]))
    data.append((idCar, Lisense, Mark, Model, Insurance_idInsurance, ChildSeat))
    # print(data[i])
    
print("Q: ", query)
print("D: ", data[0])

# insert_many_rows(query, data)

Q:  INSERT INTO TaxiTest.car (idCar, Lisense, Mark, Model, Insurance_idInsurance, ChildSeat) VALUES (%s, %s, %s, %s, %s, %s)
D:  (1, '5FLUNKP', 'Jeep', 'Renegade', 13, 1)


## Table `driver` 

*** Changes ***
~~~~python
# Add isWorking to check if driver working or not
cursor.execute("ALTER TABLE " + table + " ADD COLUMN isWorking INT NOT NULL")
cursor.execute("ALTER TABLE " + table + " MODIFY COLUMN Name VARCHAR(45) NOT NULL")
~~~~

In [21]:
table = DB_NAME + '.' + 'driver' 
get_table_review(table)
count_rows(table)

idDriver, int(11), NO, PRI, None, 
Name, varchar(45), NO, , None, 
DriverLisence, varchar(45), NO, UNI, None, 
Phone, varchar(45), NO, UNI, None, 
isWorking, int(11), NO, , None, 
Columns: ( idDriver, Name, DriverLisence, Phone, isWorking )
query = "INSERT INTO TaxiTest.driver (idDriver, Name, DriverLisence, Phone, isWorking) VALUES (%s, %s, %s, %s, %s)"
Total number of rows TaxiTest.driver: [(75,)].


75

In [24]:
total_number = 75
query = "INSERT INTO TaxiTest.driver (idDriver, Name, DriverLisence, Phone, isWorking) VALUES (%s, %s, %s, %s, %s)"

# Get all possible names_surnames combinations
names = []
with open("Names/Names.json", 'r') as f:
    names = json.load(f)
    names = list(np.random.choice(names, size=total_number))
    
# Generate UNIQUE driver licenses
licenses = set()
while len(licenses) != total_number:
    part1 = ''.join(random.choice(string.ascii_uppercase) for _ in range(7))
    part2 = ''.join(random.choice(string.digits) for _ in range(4))
    licenses.add(part1 + part2)
licenses = list(licenses)

data = []
for i in range(total_number):
    idDriver = i + 1
    # Failed processing format-parameters; Python 'str_' cannot be converted to a MySQL type
    Name = str(names[i])
    DriverLisence = licenses[i]
    Phone = '+7({}){:07}'.format(np.random.randint(50, 990), np.random.randint(0, 9999999))
    isWorking = int(np.random.choice([0, 1], p=[0.15, 0.85]))
    data.append((idDriver, Name, DriverLisence, Phone, isWorking))
    
print("Q: ", query)
print("D: ", data[0])

# insert_many_rows(query, data)

Q:  INSERT INTO TaxiTest.driver (idDriver, Name, DriverLisence, Phone, isWorking) VALUES (%s, %s, %s, %s, %s)
D:  (1, 'Осипов Альберт', 'XHWIWQG7816', '+7(820)5254510', 1)


## Table `address`

*** Changes ***
~~~~python
# Drop GeoPoint
cursor.execute("ALTER TABLE " + table + " DROP COLUMN GeoPoint")
# Delete index on UNIQUE city
cursor.execute("ALTER TABLE " + table + " DROP INDEX City_UNIQUE")
~~~~

In [25]:
table = DB_NAME + '.' + 'address' 
get_table_review(table)
count_rows(table)

idAddress, int(11), NO, PRI, None, 
City, varchar(45), NO, , None, 
Street, varchar(45), NO, , None, 
HouseNumber, int(11), NO, , None, 
Columns: ( idAddress, City, Street, HouseNumber )
query = "INSERT INTO TaxiTest.address (idAddress, City, Street, HouseNumber) VALUES (%s, %s, %s, %s)"
Total number of rows TaxiTest.address: [(1000,)].


1000

In [ ]:
total_number = 1000
query = "INSERT INTO TaxiTest.address (idAddress, City, Street, HouseNumber) VALUES (%s, %s, %s, %s)"

City = "Kazan"
# Get all streets
with open('Streets.json') as f:
    streets = json.load(f)
    streets = np.random.choice(streets, total_number, replace=False)

# Get names
data = []
for i in range(total_number):
    idAddress = i + 1
    Street = str(streets[i])
    HouseNumber = int(random.randint(1, 100))
    data.append((idAddress, City, Street, HouseNumber))
    
print("Q: ", query)
print("D: ", data[0])

# insert_many_rows(query, data)

## Table `taxi`

*** Changes ***
~~~~python
# Rename
cursor.execute("ALTER TABLE " + table + " CHANGE LatUpdate LastUpdate DATETIME")
# Change datatatype in case Python won`t permit writing to TINYINT
cursor.execute("ALTER TABLE " + table + " MODIFY COLUMN Available INT")

# Change to AUTO-INCREMENT
cursor.execute("ALTER TABLE " + table + " MODIFY COLUMN idTaxi INT AUTO_INCREMENT NOT NULL")
~~~~
*** Late: cannot change column 'idtaxi': used in a foreign key constraint 'fk_Order_Taxi' of table 'taxitest.order' ***

In [26]:
table = DB_NAME + '.' + 'Driver'
get_table_review(table, True)
table = DB_NAME + '.' + 'Car'
get_table_review(table, True)

table = "Address"
address_rows_amount = count_rows("Address")
print("")

# Try to select values from tables
table = DB_NAME + '.' + 'Taxi'
get_table_review(table)
address_rows_amount = count_rows(table)

Columns: ( idDriver, Name, DriverLisence, Phone, isWorking )
Columns: ( idCar, Lisense, Mark, Model, Insurance_idInsurance, ChildSeat )
Total number of rows Address: [(1000,)].

idTaxi, int(11), NO, MUL, None, 
Driver_idDriver, int(11), NO, PRI, None, 
Car_idCar, int(11), NO, PRI, None, 
Current_idAddress, int(11), NO, MUL, None, 
Available, int(11), YES, , None, 
LastUpdate, datetime, YES, , None, 
Columns: ( idTaxi, Driver_idDriver, Car_idCar, Current_idAddress, Available, LastUpdate )
query = "INSERT INTO TaxiTest.Taxi (idTaxi, Driver_idDriver, Car_idCar, Current_idAddress, Available, LastUpdate) VALUES (%s, %s, %s, %s, %s, %s)"
Total number of rows TaxiTest.Taxi: [(500,)].


*** If driver is not working -> taxi is not available ***

*** Taxi may be not available even if driver is not deleted *** 

In [41]:
table = "TaxiTest.Taxi"

# Using Taxi table attributes order to create query.
# INSERT INTO Taxi (idTaxi, Driver_idDriver, Car_idCar, Current_idAddress, Available, LatUpdate)
address_rows_amount = count_rows("Address")
q = "SELECT idDriver, isWorking, idCar, ROUND(1 + RAND() * %s) as Current_idAddress FROM Driver JOIN Car " % (address_rows_amount - 1)

total_amount = 500
result = select_rows(q)

taxi_combinations_set = np.arange(1, len(result))
np.random.shuffle(taxi_combinations_set)                                  
                                  
q = "INSERT INTO TaxiTest.Taxi (idTaxi, Driver_idDriver, Car_idCar, Current_idAddress, Available, LastUpdate) VALUES (%s, %s, %s, %s, %s, %s)"
data = []

for i in range(total_amount):
    row = result[taxi_combinations_set[i]]
    isWorking = row[1]
    LastUpdate = datetime.now() + timedelta(hours=np.random.randint(1, 12), minutes=np.random.randint(5, 60))
    if isWorking == 0:
        LastUpdate -= timedelta(days=np.random.randint(4, 90))
    available = int(np.random.choice([0, 1], p=[0.4, 0.6]) if isWorking else 0)
    # Order: (idTaxi, Driver_idDriver, Car_idCar, Current_idAddress, Available, LastUpdate)
    data.append((i + 1, int(row[0]), int(row[2]), int(row[3]), available, LastUpdate))
    #print(data[i])
    
# insert_many_rows(q, data)

Total number of rows Address: [(1000,)].


## Table `Operator`

In [42]:
table = DB_NAME + '.' + 'operator' 
get_table_review(table)
count_rows(table)

idOperator, int(11), NO, PRI, None, 
Name, varchar(45), NO, , None, 
Phone, varchar(45), NO, UNI, None, 
Mail, varchar(45), NO, , None, 
Columns: ( idOperator, Name, Phone, Mail )
query = "INSERT INTO TaxiTest.operator (idOperator, Name, Phone, Mail) VALUES (%s, %s, %s, %s)"
Total number of rows TaxiTest.operator: [(50,)].


50

In [43]:
total_number = 50
query = "INSERT INTO TaxiTest.operator (idOperator, Name, Phone, Mail) VALUES (%s, %s, %s, %s)"

# Get all possible names_surnames combinations
names = []
with open("Names/Names.json", 'r') as f:
    names = json.load(f)
    names = list(np.random.choice(names, size=total_number))

# Generate UNIQUE phones
phones = set()
while len(phones) != total_number:
    phone = '+7(843)' + '{:07}'.format(np.random.randint(0, 9999999))
    phones.add(phone)
phones = list(phones)

symbols = (u"абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ",
           u"abvgdeejzijklmnoprstufhzcss_y_euaABVGDEEJZIJKLMNOPRSTUFHZCSS_Y_EUA")
tr = {ord(a):ord(b) for a, b in zip(*symbols)}

data = []
for i in range(total_number):
    idOperator = i + 1
    name = names[i].replace(' ', '.')
    mail = name.translate(tr).lower() + '@gmail.com'
    data.append((idOperator, str(names[i]), str(phones[i]), str(mail)))
    
print("Q: ", query)
print("D: ", data[0])

# insert_many_rows(query, data)

Q:  INSERT INTO TaxiTest.operator (idOperator, Name, Phone, Mail) VALUES (%s, %s, %s, %s)
D:  (1, 'Федоров Вадим', '+7(843)2283950', 'fedorov.vadim@gmail.com')


## Table `Client`

*** Changes ***
~~~~python
cursor.execute("ALTER TABLE " + table + " CHANGE COLUMN Name Name VARCHAR(45) AFTER idClient")
~~~~

In [44]:
table = DB_NAME + '.' + 'client' 
get_table_review(table)
count_rows(table)

idClient, int(11), NO, PRI, None, auto_increment
Name, varchar(45), YES, , None, 
Phone, varchar(45), NO, UNI, None, 
Password, varchar(45), NO, , None, 
Columns: ( idClient, Name, Phone, Password )
query = "INSERT INTO TaxiTest.client (idClient, Name, Phone, Password) VALUES (%s, %s, %s, %s)"
Total number of rows TaxiTest.client: [(5000,)].


5000

In [45]:
total_number = 5000
query = "INSERT INTO TaxiTest.client (idClient, Name, Phone, Password) VALUES (%s, %s, %s, %s)"

# Get all possible names_surnames combinations
names = []
with open("Names/Names.json", 'r') as f:
    names = json.load(f)
    names = list(np.random.choice(names, size=total_number))

names = np.random.choice(names, total_number, replace=False)

# Generate UNIQUE phones
phones = set()
while len(phones) != total_number:
    phone = '+7({:03}){:07}'.format(np.random.randint(50, 990), np.random.randint(0, 9999999))
    phones.add(phone)
phones = list(phones)

alphanums = string.ascii_letters + string.digits
    
data = []
for i in range(total_number):
    idClient = i + 1
    password = ''.join(random.choice(alphanums) for _ in range(7))
    data.append((idClient, str(names[i]), phones[i], password))
    
print("Q: ", query)
print("D: ", data[0])

# insert_many_rows(query, data)

Q:  INSERT INTO TaxiTest.client (idClient, Name, Phone, Password) VALUES (%s, %s, %s, %s)
D:  (1, 'Лебедев Виктор', '+7(950)6118323', 'WkUQVNw')


## Table `PhoneCall`

*** Changes ***
~~~~python
# Unnset NOT NULL
cursor.execute("ALTER TABLE " + table + " CHANGE COLUMN `Duration` `Duration` INT AFTER Datetime")
# Change Primary Key from (idOperator, idClient) to idCall
cursor.execute("ALTER TABLE " + table + " DROP PRIMARY KEY, ADD PRIMARY KEY (idCall)")

# Rename to remove MySQL Errors
cursor.execute("ALTER TABLE " + table + " CHANGE COLUMN `datetime` `datetimeValue` DATETIME")
# Rename because Call is a system word
cursor.execute("ALTER TABLE " + table + " RENAME TO PhoneCall")
# Replace
cursor.execute("ALTER TABLE " + table + " CHANGE COLUMN `Datetime` `Datetime` Datetime AFTER Client_idClient")
~~~~
*** Change idCall to NOT AUTO-INCREMENT ***
~~~~sql
SHOW CREATE TABLE PhoneCall;
ALTER TABLE `Order` DROP FOREIGN KEY fk_Order_Call;
ALTER TABLE `PhoneCall` CHANGE COLUMN `idCall` `idCall` INT NOT NULL;
# Adding PRIMARY KEY on idCall not needed in case it already exists 

# Return CONSTRAINT (Workbench): Used inital Create script to identify constraint and FK names.
ALTER TABLE `Order` ADD CONSTRAINT fk_Order_Call FOREIGN KEY (`Call_idCall`) REFERENCES `PhoneCall`(`idCall`);

# Check constraint
SELECT TABLE_NAME,COLUMN_NAME,CONSTRAINT_NAME, REFERENCED_TABLE_NAME,REFERENCED_COLUMN_NAME
FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE
WHERE
  REFERENCED_TABLE_SCHEMA = 'TaxiTest' AND
  REFERENCED_TABLE_NAME = 'PhoneCall' AND
  REFERENCED_COLUMN_NAME = 'idCall';

~~~~

In [16]:
table = DB_NAME + '.' + 'PhoneCall'
get_table_review(table)

count_rows(table)

idCall, int(11), NO, PRI, None, 
Operator_idOperator, int(11), NO, MUL, None, 
Client_idClient, int(11), NO, MUL, None, 
datetimeValue, datetime, YES, , None, 
Duration, int(11), YES, , None, 
Columns: ( idCall, Operator_idOperator, Client_idClient, datetimeValue, Duration )
query = "INSERT INTO TaxiTest.PhoneCall (idCall, Operator_idOperator, Client_idClient, datetimeValue, Duration) VALUES (%s, %s, %s, %s, %s)"
Total number of rows TaxiTest.PhoneCall: [(1500,)].


1500

In [46]:
'''
query = "SELECT idOperator, idClient FROM Operator JOIN Client"
result = select_rows(table, q)
client_max_id * operators_max_id = len(result)
'''
# This query is hard, cause handles 5000 * 50 combinations -> 
# execute with sumpler queries (need only range of corresponding ids)

operators_max_id = count_rows("Operator")
client_max_id = count_rows("Client")

total_amount = 1500
all_combinations = list(itertools.product(np.arange(1, operators_max_id), np.arange(1, client_max_id)))
np.random.shuffle(all_combinations)

# Roolback on datetime then
query = "INSERT INTO PhoneCall (idCall, Operator_idOperator, Client_idClient, datetimeValue, Duration) VALUES (%s, %s, %s, %s, %s)"
data = []

print("Q: ", query)
with open('C:/SQL/PhoneCall - INSERT 1500 with DATETIME.sql', 'w') as f:
    f.write('INSERT INTO PhoneCall (idCall, Operator_idOperator, Client_idClient, datetimeValue, Duration) VALUES \n')
    for i in range(total_amount):
        idCall = i + 1
        (Operator_idOperator, Client_idClient) = all_combinations[i]
        duration = int(np.random.randint(20, 120))
        # Order: Operator_idOperator, Client_idClient, Duration  
        datetime_value = datetime(2018, np.random.randint(1, 8), np.random.randint(1, 28)) + timedelta(minutes=np.random.randint(-1000, 1000))
        formatted_date = datetime_value.strftime('%Y-%m-%d %H:%M:%S')
        data.append((idCall, Operator_idOperator, Client_idClient, formatted_date, duration))
        f.write("(%s, %s, %s, \'%s\', %s),\n" % data[i])
    
print("Q: ", query)
print("D: ", data[0])
# insert_many_rows(query, data)

Total number of rows Operator: [(50,)].
Total number of rows Client: [(5000,)].
Q:  INSERT INTO PhoneCall (idCall, Operator_idOperator, Client_idClient, datetimeValue, Duration) VALUES (%s, %s, %s, %s, %s)
Q:  INSERT INTO PhoneCall (idCall, Operator_idOperator, Client_idClient, datetimeValue, Duration) VALUES (%s, %s, %s, %s, %s)
D:  (1, 16, 2012, '2018-03-05 09:43:00', 57)


*** Result: *** written to text file and inserted using MySQL Workbench. 

*** Update: *** fixed function call

## Table `PaymentService`

In [47]:
table = DB_NAME + '.' + 'PaymentService'
get_table_review(table)
count_rows(table)

idPaymentService, int(11), NO, PRI, None, 
Name, varchar(45), NO, UNI, None, 
Address, varchar(45), NO, UNI, None, 
Site, varchar(45), NO, UNI, None, 
Columns: ( idPaymentService, Name, Address, Site )
query = "INSERT INTO TaxiTest.PaymentService (idPaymentService, Name, Address, Site) VALUES (%s, %s, %s, %s)"
Total number of rows TaxiTest.PaymentService: [(13,)].


13

In [50]:
with open('PaymentService/Service.json', 'r') as f:
    services = json.load(f)
total_amount = len(services)

with open('Street/Street.json', 'r') as f:
    streets = json.load(f)
    streets = np.random.choice(streets, total_amount)
    
query = "INSERT INTO `paymentservice` (idPaymentService, Name, Address, Site) VALUES (%s, %s, %s, %s)"
print(query)
data = []    
for i in range(total_amount):
    idPaymentService = i + 1
    Name = services[i]
    Address = 'Kazan, ' + streets[i] + ', ' + str(np.random.randint(1, 10))
    Site = Name.lower()[:-10] + '.com'    
    data.append((idPaymentService, Name, Address, Site))
    
print("Q: ", query)
print("D: ", data[0])
print("")

#insert_many_rows(query, data)

INSERT INTO `paymentservice` (idPaymentService, Name, Address, Site) VALUES (%s, %s, %s, %s)
Q:  INSERT INTO `paymentservice` (idPaymentService, Name, Address, Site) VALUES (%s, %s, %s, %s)
D:  (1, 'AlfaPayService', 'Kazan, Belogorsk, 3', 'alfa.com')



~~~~sql
INSERT INTO TaxiTest.PaymentService (idPaymentService, Name, Address, Site) VALUES
(1, 'SberbankPayService', 'Kazan, Baltic, 4', 'sberbank.com') ,
(2, 'VTBPayService', 'Kazan, Peter, 2', 'vtb.com') ,
(3, 'GazprombankPayService', 'Kazan, Kirov, 6', 'gazprombank.com') ,
(4, 'VTB-24PayService', 'Kazan, Ichigovaya, 8', 'vtb-24.com') ,
(5, 'RosselkhozbankPayService', 'Kazan, Syzranskaya, 8', 'rosselkhozbank.com') ,
(6, 'BankPayService', 'Kazan, Aktaş, 1', 'bank.com') ,
(7, 'AlfaPayService', 'Kazan, Winter, 1', 'alfa.com') ,
(8, 'NomosPayService', 'Kazan, Armuti, 8', 'nomos.com') ,
(9, 'UnicreditPayService', 'Kazan, Cross-Road, 1', 'unicredit.com') ,
(10, 'RosbankPayService', 'Kazan, Kyrlay, 9', 'rosbank.com') ,
(11, 'PromsvyazbankPayService', 'Kazan, Snowball, 2', 'promsvyazbank.com') ,
(12, 'RaiffeisenbankPayService', 'Kazan, Bastion, 7', 'raiffeisenbank.com');
~~~~


## Table `Payment`

*** Changes ***
~~~~python
# Change to Cash (value 0) or Card (value 1)
cursor.execute("ALTER TABLE Payment MODIFY Type INT NOT NULL AFTER PayTime")

# Drop FOREIGN KEY 
cursor.execute("ALTER TABLE Payment DROP FOREIGN KEY fk_Payment_PaymentService1")

# Change COLUMN PaymentService_idPaymentService for NULL values insertion opportunity
cursor.execute("ALTER TABLE Payment CHANGE PaymentService_idPaymentService PaymentService_idPaymentService INT AFTER PayTime")

# Return the KEY, now available to insert with NULL values
cursor.execute("ALTER TABLE Payment ADD FOREIGN KEY fk_Payment_PaymentService1 (PaymentService_idPaymentService) REFERENCES PaymentService(idPaymentService) ")

# Ruble coins are not used at all, DOUBLE type is not necessary 
cursor.execute("ALTER TABLE Payment MODIFY Cost INT NOT NULL ")

# Rename to exclude mistakes
cursor.execute("ALTER TABLE Payment CHANGE Type PaymentType INT NOT NULL ")

# Delete Cost Column cause it is already in the Order
cursor.execute("ALTER TABLE Payment DROP COLUMN COST")
~~~~
~~~~sql
# Add explicitly one row to table PaymentService with idPaymentService = 0
INSERT INTO `paymentservice` (idPaymentService, Name, Address, Site) VALUES (0, 'CashService', 'NoAddress', 'NoSite');

~~~~

In [51]:
table = DB_NAME + '.' + 'Payment'
get_table_review(table)
count_rows(table)

idPayment, int(11), NO, PRI, None, auto_increment
PayTime, datetime, YES, , None, 
PaymentType, int(11), NO, , None, 
PaymentService_idPaymentService, int(11), YES, MUL, None, 
Paycheck, varchar(45), YES, , None, 
Columns: ( idPayment, PayTime, PaymentType, PaymentService_idPaymentService, Paycheck )
query = "INSERT INTO TaxiTest.Payment (idPayment, PayTime, PaymentType, PaymentService_idPaymentService, Paycheck) VALUES (%s, %s, %s, %s, %s)"
Total number of rows TaxiTest.Payment: [(1200,)].


1200

## Table `Orders`

*** Changes ***
~~~~python
# Order is a keyword
cursor.execute("ALTER TABLE Taxitest.Order RENAME Orders")
~~~~

In [52]:
table = DB_NAME + '.' + 'Orders'
get_table_review(table)
count_rows(table)

idOrder, int(11), NO, PRI, None, 
Call_idCall, int(11), NO, MUL, None, 
Departure_idAddress, int(11), NO, MUL, None, 
Destination_idAddress, int(11), NO, MUL, None, 
Taxi_idTaxi, int(11), NO, MUL, None, 
Payment_idPayment, int(11), NO, MUL, None, 
StartDatetime, datetime, NO, , None, 
FinishDatetime, datetime, YES, , None, 
Cost, int(11), NO, , None, 
Rate, int(11), YES, , None, 
Columns: ( idOrder, Call_idCall, Departure_idAddress, Destination_idAddress, Taxi_idTaxi, Payment_idPayment, StartDatetime, FinishDatetime, Cost, Rate )
query = "INSERT INTO TaxiTest.Orders (idOrder, Call_idCall, Departure_idAddress, Destination_idAddress, Taxi_idTaxi, Payment_idPayment, StartDatetime, FinishDatetime, Cost, Rate) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
Total number of rows TaxiTest.Orders: [(1200,)].


1200

In [53]:
# Try to extract/insert datetime values
q = "SELECT idCall, datetimeValue FROM PhoneCall"
phone_call_data = select_rows(q)
for i in range(len(phone_call_data)):
    print(phone_call_data[i])
    t = phone_call_data[i][1]
    print(t + timedelta(minutes=50))
    break

(1, datetime.datetime(2018, 6, 15, 4, 29))
2018-06-15 05:19:00


In [54]:
from numpy.random import randint
from numpy.random import choice

MAX_PAYMENT_SERVICE_ID = count_rows("PaymentService")
MAX_CALL_ID = count_rows('PhoneCall')
MAX_ADDRESS_ID = count_rows('Address')
MAX_TAXI_ID = count_rows('Taxi')

TOTAL_ORDERS_AMOUNT = 1200

order_from_call = choice(np.arange(1, len(phone_call_data)), TOTAL_ORDERS_AMOUNT, replace=False)
order_from_call = sorted(order_from_call)

f_order = open('C:/SQL/Orders.sql', 'w')    
query_order = "INSERT INTO Orders (idOrder, Call_idCall, Departure_idAddress, Destination_idAddress, Taxi_idTaxi, Payment_idPayment, StartDatetime, FinishDatetime, Cost, Rate) VALUES\n"
f_order.write(query_order)
data_order = []

f_payment = open('C:/SQL/Payment.sql', 'w')
query_payment = "INSERT INTO Payment (idPayment, PaymentType, PaymentService_idPaymentService, Paycheck) VALUES \n"
f_payment.write(query_payment)
data_payment = []

q = "SELECT idCall, datetimeValue FROM PhoneCall ORDER BY datetimeValue"
phone_call_data = select_rows(q)

for i in range(TOTAL_ORDERS_AMOUNT):
    idOrder = i + 1
    idPayment = i + 1
    t = order_from_call[i]
    Call_idCall = phone_call_data[t][0]
    StartDatetime = phone_call_data[t][1] + timedelta(minutes=randint(5, 360))

    (Departure_idAddress, Destination_idAddress) = choice(np.arange(1, MAX_ADDRESS_ID), 2, replace=False) 
    Taxi_idTaxi = randint(1, MAX_TAXI_ID)
    FinishDatetime = StartDatetime + timedelta(minutes=randint(5, 45))
    Cost = int(6 * (FinishDatetime - StartDatetime).total_seconds() // 60)
    Rate = choice([3,4,5], p=[0.2, 0.3, 0.5])
    
    order_tuple = (idOrder, Call_idCall, Departure_idAddress, Destination_idAddress, Taxi_idTaxi, idPayment, StartDatetime, FinishDatetime, Cost, Rate)
    row_order = "(%s, %s, %s, %s, %s, %s, \'%s\',\'%s\', %s, %s),\n"
 
    # Payment
    payment_type = choice([0, 1], p=[0.65, 0.35]) # ['Card', 'Cash']
    if payment_type == 0:
        id_payment_service = randint(1, MAX_PAYMENT_SERVICE_ID)
        paycheck = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(10))
        row_payment = "(%s, %s, %s, \"%s\"),\n"
    else:
        id_payment_service = 'NULL'
        paycheck = 'NULL'
        row_payment = "(%s, %s, %s, %s),\n"
    payment_tuple = (idPayment, payment_type, id_payment_service, paycheck)
    
    if i == TOTAL_ORDERS_AMOUNT - 1:
        row_order = row_order[:-2] + ';'
        row_payment = row_payment[:-2] + ';'
    
    f_order.write(row_order % order_tuple)
    f_payment.write(row_payment % payment_tuple)
    
f_order.close()
f_payment.close()

print("\nQ: %sD: %s" % (query_order, order_tuple))
print("\nQ: %sD: %s" % (query_payment, payment_tuple))

Total number of rows PaymentService: [(13,)].
Total number of rows PhoneCall: [(1500,)].
Total number of rows Address: [(1000,)].
Total number of rows Taxi: [(500,)].

Q: INSERT INTO Orders (idOrder, Call_idCall, Departure_idAddress, Destination_idAddress, Taxi_idTaxi, Payment_idPayment, StartDatetime, FinishDatetime, Cost, Rate) VALUES
D: (1200, 688, 633, 279, 13, 1200, datetime.datetime(2018, 7, 27, 18, 55), datetime.datetime(2018, 7, 27, 19, 26), 186, 5)

Q: INSERT INTO Payment (idPayment, PaymentType, PaymentService_idPaymentService, Paycheck) VALUES 
D: (1200, 0, 10, 'MRMEGGVQNL')


# SQL запросы

In [57]:
# DESCRIBE taxitest.order;
def get_query_result(q):
    result = select_rows(q)
    for i in range(len(result)):
        print(result[i])
        if i > 10:
            break

cursor.execute("USE " + DB_NAME)
cursor.execute("SHOW Tables")
tables = cursor.fetchall()


for table in tables:
    print("Table:", str(table[0]))
    get_table_review(table[0], only_columns_names=True)


Table: address
Columns: ( idAddress, City, Street, HouseNumber )
Table: car
Columns: ( idCar, Lisense, Mark, Model, Insurance_idInsurance, ChildSeat )
Table: client
Columns: ( idClient, Name, Phone, Password )
Table: driver
Columns: ( idDriver, Name, DriverLisence, Phone, isWorking )
Table: insurance
Columns: ( idInsurance, InsCompany_idInsCompany, InsuranceStartDate, InsuranceEndDate )
Table: insurancecompany
Columns: ( idInsuranceCompany, Address, Phone, Mail, Name )
Table: operator
Columns: ( idOperator, Name, Phone, Mail )
Table: orders
Columns: ( idOrder, Call_idCall, Departure_idAddress, Destination_idAddress, Taxi_idTaxi, Payment_idPayment, StartDatetime, FinishDatetime, Cost, Rate )
Table: payment
Columns: ( idPayment, PayTime, PaymentType, PaymentService_idPaymentService, Paycheck )
Table: paymentservice
Columns: ( idPaymentService, Name, Address, Site )
Table: phonecall
Columns: ( idCall, Operator_idOperator, Client_idClient, datetimeValue, Duration )
Table: taxi
Columns: ( i

### Список всех свободных такси

In [58]:
q = "SELECT * FROM TaxiTest.Taxi WHERE Taxi.Available = 1"
print("idTaxi, Driver_idDriver, Car_idCar, Current_idAddress, Available, LatUpdate")
get_query_result(q)

idTaxi, Driver_idDriver, Car_idCar, Current_idAddress, Available, LatUpdate
(338, 1, 23, 593, 1, datetime.datetime(2018, 9, 17, 22, 26, 20))
(164, 1, 89, 610, 1, datetime.datetime(2018, 9, 17, 19, 53, 20))
(463, 2, 7, 514, 1, datetime.datetime(2018, 9, 17, 19, 37, 20))
(379, 2, 34, 94, 1, datetime.datetime(2018, 9, 17, 22, 13, 20))
(30, 2, 92, 364, 1, datetime.datetime(2018, 9, 18, 0, 54, 20))
(121, 3, 2, 509, 1, datetime.datetime(2018, 9, 18, 4, 12, 20))
(215, 3, 51, 416, 1, datetime.datetime(2018, 9, 17, 19, 41, 20))
(348, 3, 67, 529, 1, datetime.datetime(2018, 9, 17, 22, 58, 20))
(151, 3, 99, 838, 1, datetime.datetime(2018, 9, 18, 3, 24, 20))
(38, 4, 31, 389, 1, datetime.datetime(2018, 9, 17, 20, 59, 20))
(75, 4, 53, 568, 1, datetime.datetime(2018, 9, 17, 18, 50, 20))
(416, 4, 72, 128, 1, datetime.datetime(2018, 9, 18, 4, 22, 20))


### Список всех свободных такси с указанием авто

In [59]:
q = "SELECT idTaxi, Mark, Model, Lisense, ChildSeat FROM TaxiTest.Taxi JOIN Car ON Car_idCar = idCar WHERE Taxi.Available = 1"

print("idTaxi, Mark, Model, Lisense, ChildSeat (0/1)")
get_query_result(q)

idTaxi, Mark, Model, Lisense, ChildSeat (0/1)
(338, 'Citroen', 'Berlingo', '3EGBSVS', 0)
(164, 'Renault', 'Trafic', 'W5BINWW', 1)
(463, 'Honda', 'CR-V', 'ZB12ELJ', 0)
(379, 'Citroen', 'C1', '3OGNBSE', 1)
(30, 'Renault', 'Kangoo', '04HS9CT', 0)
(121, 'Mercedes', 'GLA', 'ZMTSOWR', 0)
(215, 'Peugeot', 'Partner', 'N2PVBQQ', 1)
(348, 'Volkswagen', 'Transporter', 'AWNK219', 0)
(151, 'Mercedes', 'Sprinter', 'NULSZK7', 0)
(38, 'Skoda', 'Octavia', '8HC0G2Z', 1)
(75, 'Toyota', 'RAV4', '6025U7Z', 1)
(416, 'Peugeot', '3008', '0NDF6UE', 1)


### Список всех клиентов, которых обслуживал авто (входной параметр – номер Авто)

In [65]:
q = "SELECT Client_idClient, Phone, Name FROM (\
        SELECT DISTINCT Client_idClient FROM Client, PhoneCall JOIN Orders \
        ON PhoneCall.idCall = Orders.Call_idCall WHERE Orders.Taxi_idTaxi IN (\
            SELECT idTaxi FROM Taxi JOIN Car ON Taxi.Car_idCar = Car.idCar \
            WHERE Car.Lisense = 'ZMTSOWR'\
        )\
    ) AS T, Client WHERE Client.idClient = T.Client_idClient"

print("idClient, Phone, Name")
get_query_result(q)

idClient, Phone, Name
(2736, '+7(951)4491502', 'Святослав Комаров')
(1239, '+7(841)3053157', 'Лев Воронин')
(1147, '+7(875)7677765', 'Мирослав Коновалов')
(1041, '+7(761)5678575', 'Рустам Кузин')
(4363, '+7(209)2277892', 'Эдуард Серов')
(3703, '+7(397)6720811', 'Семен Ларин')
(3893, '+7(589)6133903', 'Валерий Иванов')
(2574, '+7(210)1375577', 'Ильяс Емельянов')
(3146, '+7(680)2392000', 'Григорий Молчанов')
(4795, '+7(614)7039251', 'Виктор Моисеев')
(1478, '+7(726)5952862', 'Ильяс Прокофьев')
(2731, '+7(773)0999344', 'Константин Михайлов')


### Список всех авто, которыми пользовался клиент в интервале времени (входные параметры – телефон клиента, дата начала и дата окончания)

In [67]:
# Sort Clients by trips they had
q = "SELECT Client_idClient, COUNT(*) AS TotalCalls FROM PhoneCall \
     GROUP BY Client_idClient ORDER BY TotalCalls DESC"

print("Client_idClient, COUNT(Client`s trips amount)")
get_query_result(q)

Client_idClient, COUNT(Client`s trips amount)
(1385, 4)
(1727, 4)
(3476, 3)
(325, 3)
(1060, 3)
(3893, 3)
(1164, 3)
(1815, 3)
(2991, 3)
(3325, 3)
(150, 3)
(768, 3)


In [66]:
# Sort Clients by trips they had and get thir names and phones
q = "SELECT Client_idClient, Name, Phone, TotalCalls FROM Client, (\
        SELECT Client_idClient, COUNT(*) AS TotalCalls FROM PhoneCall GROUP BY Client_idClient\
    ) AS T WHERE idClient = Client_idClient ORDER BY TotalCalls DESC"

print("Client_idClient, Name, Phone, TotalCalls")
get_query_result(q)

Client_idClient, Name, Phone, TotalCalls
(1385, 'Гордей Николаев', '+7(726)5384089', 4)
(1727, 'Демид Левин', '+7(376)6414991', 4)
(3476, 'Михаил Ларин', '+7(314)0212667', 3)
(325, 'Рамиль Кириллов', '+7(537)2085180', 3)
(1060, 'Станислав Бородин', '+7(929)6680229', 3)
(3893, 'Валерий Иванов', '+7(589)6133903', 3)
(1164, 'Станислав Кириллов', '+7(226)3091976', 3)
(1815, 'Егор Соловьев', '+7(721)5296773', 3)
(2991, 'Игнат Юдин', '+7(382)0077846', 3)
(3325, 'Геннадий Дроздов', '+7(421)0561994', 3)
(150, 'Дмитрий Фадеев', '+7(526)5981637', 3)
(768, 'Никита Герасимов', '+7(381)7810300', 3)


In [68]:
# Get Orders of Client with idClient = 1385
q = "SELECT idOrder FROM PhoneCall Left JOIN Orders ON PhoneCall.idCall = Call_idCall WHERE Client_idClient = 1385"

print("idOrder")
get_query_result(q)

idOrder
(796,)
(313,)
(16,)
(64,)


In [69]:
# Just checking the Phone and idClient accordance 
q = "SELECT idClient FROM Client WHERE Client.Phone='+7(726)5384089'"
get_query_result(q)

(1385,)


In [72]:
# Get detailed list of orders
q = "SELECT idOrder, Call_idCall, Taxi_idTaxi, StartDatetime, FinishDatetime, Cost, Rate FROM Orders WHERE idOrder IN (\
        SELECT idOrder FROM PhoneCall Left JOIN Orders ON PhoneCall.idCall = Orders.Call_idCall WHERE Client_idClient IN (\
            SELECT idClient FROM Client WHERE Phone='+7(726)5384089' \
        )\
    )"
print("idOrder, Call_idCall, Taxi_idTaxi, StartDatetime, FinishDatetime, Cost, Rate")
get_query_result(q)

idOrder, Call_idCall, Taxi_idTaxi, StartDatetime, FinishDatetime, Cost, Rate
(796, 96, 57, datetime.datetime(2018, 5, 20, 12, 33), datetime.datetime(2018, 5, 20, 12, 38), 30, 5)
(313, 536, 310, datetime.datetime(2018, 3, 1, 12, 4), datetime.datetime(2018, 3, 1, 12, 21), 102, 5)
(16, 646, 224, datetime.datetime(2018, 1, 2, 17, 20), datetime.datetime(2018, 1, 2, 17, 56), 216, 5)
(64, 1311, 44, datetime.datetime(2018, 1, 12, 17, 59), datetime.datetime(2018, 1, 12, 18, 35), 216, 4)


In [70]:
# Get detailed list of orders with sort on datetime
q = "SELECT idOrder, Call_idCall, Taxi_idTaxi, StartDatetime, FinishDatetime, Cost, Rate\
     FROM Orders WHERE idOrder IN (\
        SELECT idOrder FROM PhoneCall Left JOIN Orders ON PhoneCall.idCall = Call_idCall WHERE Client_idClient IN (\
            SELECT idClient FROM Client WHERE Phone='+7(726)5384089' \
        )\
    ) AND StartDatetime > '2018-01-03 00:00:00' AND FinishDatetime < '2018-05-01 00:00:00' ORDER BY StartDatetime"

print("idOrder, Call_idCall, Taxi_idTaxi, StartDatetime, FinishDatetime, Cost, Rate")
get_query_result(q)

idOrder, Call_idCall, Taxi_idTaxi, StartDatetime, FinishDatetime, Cost, Rate
(64, 1311, 44, datetime.datetime(2018, 1, 12, 17, 59), datetime.datetime(2018, 1, 12, 18, 35), 216, 4)
(313, 536, 310, datetime.datetime(2018, 3, 1, 12, 4), datetime.datetime(2018, 3, 1, 12, 21), 102, 5)


In [71]:
# Get detailed list of orders with sort on datetime with CarId
q = "SELECT idOrder, Call_idCall, Taxi_idTaxi, Car_idCar, StartDatetime, FinishDatetime, Cost, Rate FROM \
        (SELECT idOrder, Call_idCall, Taxi_idTaxi, StartDatetime, FinishDatetime, Cost, Rate FROM Orders WHERE idOrder IN \
            (SELECT idOrder FROM PhoneCall Left JOIN Orders ON PhoneCall.idCall = Call_idCall WHERE Client_idClient IN \
                (SELECT idClient FROM Client WHERE Phone='+7(726)5384089')) AND StartDatetime > '2018-01-03 00:00:00' AND FinishDatetime < '2018-05-01 00:00:00' \
        ) AS T2 JOIN Taxi ON T2.Taxi_idTaxi = Taxi.idTaxi"

print("idOrder, Call_idCall, Taxi_idTaxi, Car_idCar, StartDatetime, FinishDatetime, Cost, Rate")
get_query_result(q)

idOrder, Call_idCall, Taxi_idTaxi, Car_idCar, StartDatetime, FinishDatetime, Cost, Rate
(64, 1311, 44, 17, datetime.datetime(2018, 1, 12, 17, 59), datetime.datetime(2018, 1, 12, 18, 35), 216, 4)
(313, 536, 310, 16, datetime.datetime(2018, 3, 1, 12, 4), datetime.datetime(2018, 3, 1, 12, 21), 102, 5)


In [33]:
# Get detailed list of orders with sort on datetime with CarId and cars 
# Only valuable aattributes chosed

q = "SELECT idOrder, Lisense, Model, Mark, StartDatetime, FinishDatetime, Cost, Rate FROM (\
        SELECT idOrder, Call_idCall, Taxi_idTaxi, Car_idCar, StartDatetime, FinishDatetime, Cost, Rate FROM (\
            SELECT idOrder, Call_idCall, Taxi_idTaxi, StartDatetime, FinishDatetime, Cost, Rate FROM Orders WHERE idOrder IN (\
                SELECT idOrder FROM PhoneCall Left JOIN Orders ON PhoneCall.idCall = Call_idCall WHERE Client_idClient IN (\
                    SELECT idClient FROM Client WHERE Phone='+7(726)5384089'\
                )\
            ) AND StartDatetime > '2018-01-03 00:00:00' AND FinishDatetime < '2018-05-01 00:00:00' \
        ) AS T2 JOIN Taxi ON T2.Taxi_idTaxi = Taxi.idTaxi \
    ) AS T3 JOIN Car ON Car.idCar = T3.Car_idCar"

print("idOrder, Lisense, Model, Mark, StartDatetime, FinishDatetime, Cost, Rate")
get_query_result(q)

idOrder, Lisense, Model, Mark, StartDatetime, FinishDatetime, Cost, Rate
(64, '1I9C9N5', 'CLA', 'Mercedes', datetime.datetime(2018, 1, 12, 17, 59), datetime.datetime(2018, 1, 12, 18, 35), 216, 4)
(313, 'KSIY6KD', 'Cee’d', 'Kia', datetime.datetime(2018, 3, 1, 12, 4), datetime.datetime(2018, 3, 1, 12, 21), 102, 5)


### Список всех заказов, которые обработаны оператором (входной параметр id-оператора) 


In [34]:
q = "SELECT idOrder, Cost, Rate FROM Orders, PhoneCall WHERE Orders.Call_idCall IN \
    (SELECT PhoneCall.idCall FROM PhoneCall WHERE Operator_idOperator = 10)"

print("idOrder, Cost, Rate")
get_query_result(q)

idOrder, Cost, Rate
(1061, 108, 5)
(945, 216, 4)
(1196, 90, 5)
(589, 252, 5)
(1038, 60, 4)
(714, 174, 4)
(348, 132, 5)
(109, 54, 4)
(752, 78, 4)
(856, 246, 5)
(486, 174, 3)
(435, 168, 5)


In [35]:
cursor.close()
cnx.close()

# The end.